<a href="https://colab.research.google.com/github/febin-alex/Water-Leakage-Detection-System/blob/main/MajorP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:

#prerequsite

!pip install keras-preprocessing

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.6/42.6 kB 2.5 MB/s eta 0:00:00


In [ ]:

#mounting to drive for saving dataset

import random
import csv
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:

#write to a CSV dataset

with open('dataset.csv', 'w', newline='') as file:
    writer = csv.writer(file)

#dataset header with daily water usage of 20houses + leak yes or no 1 or 0

    writer.writerow(["flow", "material", "diameter", "leak"])
    i=0
    material=0
    flow=0
    diameter=0.0
#data for 365 days
    di_list = [0.5,1]

    while(i<300):
     
      
#random values for houses from 0 to 100l

      flow=random.randint(0,100)
      material=random.randint(1,3)
      diameter=random.choice(di_list)
      
      i+=1

#output is sum of daily usage of 20 houses in an area

#if output is greater than an arbitrary valuue 1000l daily then there is leak

      if(diameter==0.5):
        if(material==1):
          if(flow<5 or flow>30):
            leak=1
          else:
            leak=0
        elif(material==2):
          if(flow<3 or flow>35):
            leak=1
          else:
            leak=0
        else:
          if(flow<1 or flow>40):
            leak=1
          else:
            leak=0
      else:
        if(material==1):
          if(flow<5 or flow>40):
            leak=1
          else:
            leak=0
        elif(material==2):
          if(flow<3 or flow>45):
            leak=1
          else:
            leak=0
        else:
          if(flow<1 or flow>50):
            leak=1
          else:
            leak=0
      

        


#write values
      
      writer.writerow([flow,material,diameter,leak])

In [ ]:

#Code for bilstm


import numpy as np
import pandas as pd
from keras_preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, LSTM, Bidirectional
from sklearn.model_selection import train_test_split

# Load the water dataset
data = pd.read_csv("dataset.csv")

# Prepare the inputs for the model
x = data.values[:,:-1]
y = data.values[:,-1]


# Remove the rows with non-numeric values
data = data.apply(pd.to_numeric, errors='coerce')
data = data.dropna()


# Preprocess the inputs
max_length = 30
x = pad_sequences(x, maxlen=max_length, dtype='float32')

# Convert the target to categorical
y = np.array(y, dtype='float32')

X_train, X_test, Y_train, Y_test = train_test_split (x, y, test_size=0.2)

# Build the model
model = Sequential()
model.add(Bidirectional(LSTM(32), input_shape=(max_length, 1)))
model.add(Dense(1, activation='sigmoid'))

# Compile the model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# Train the model
model.fit(X_train, Y_train, epochs=10, batch_size=64)

# Evaluate the model
scores = model.evaluate(X_test, Y_test, verbose=0)
print("Accuracy:", scores[1])

Epoch 1/10
4/4 [==============================] - 10s 26ms/step - loss: 0.7458 - accuracy: 0.3708
Epoch 2/10
4/4 [==============================] - 0s 23ms/step - loss: 0.7239 - accuracy: 0.3708
Epoch 3/10
4/4 [==============================] - 0s 22ms/step - loss: 0.6993 - accuracy: 0.4417
Epoch 4/10
4/4 [==============================] - 0s 22ms/step - loss: 0.6836 - accuracy: 0.6125
Epoch 5/10
4/4 [==============================] - 0s 22ms/step - loss: 0.6625 - accuracy: 0.6292
Epoch 6/10
4/4 [==============================] - 0s 23ms/step - loss: 0.6469 - accuracy: 0.6292
Epoch 7/10
4/4 [==============================] - 0s 23ms/step - loss: 0.6262 - accuracy: 0.6292
Epoch 8/10
4/4 [==============================] - 0s 23ms/step - loss: 0.6089 - accuracy: 0.6292
Epoch 9/10
4/4 [==============================] - 0s 23ms/step - loss: 0.5691 - accuracy: 0.6292
Epoch 10/10
4/4 [==============================] - 0s 24ms/step - loss: 0.5229 - accuracy: 0.7083
Accuracy: 0.800000011920929


In [ ]:
import pickle
filename='model.sav'
pickle.dump(model, open(filename,'wb'))

In [ ]:
loaded_model=pickle.load(open(filename,'rb'))
result=loaded_model.evaluate(X_test,Y_test, verbose=0)
print(result)
print("Accuracy:", scores[1])

[0.4506792724132538, 0.800000011920929]
Accuracy: 0.800000011920929
